In [25]:
# !sudo pip install h5py

The directory '/home/mohan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/mohan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#snimissingwarning.
  SNIMiss

In [1]:
import copy
import datetime
import numpy as np
import time
import pandas as pd
import pandas_datareader.data as web
from sklearn.cluster import KMeans
from sklearn import preprocessing
import plotly
import itertools
from talib.abstract import *
import os
import sys
from keras.models import model_from_json
from keras.models import model_from_yaml
import requests
import numpy as np
import pandas as pd
import h5py
from keras.models import Sequential
from keras.layers import Dense
from sklearn.externals import joblib
print("Import Function for the Neural Network Done")
from multiprocessing import Pool


# import plotly.plotly as py
# from plotly.graph_objs import *
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# plotly.tools.set_credentials_file(username='maiti.t', api_key='km8Kdfszic1Cu6ZAWiZJ')
# plotly.offline.init_notebook_mode(connected=True)

Using TensorFlow backend.


Import Function for the Neural Network Done


In [2]:
import pickle
fileDir = os.path.dirname(os.path.realpath('__file__'))
print(' Local Path defined')
print(fileDir)

 Local Path defined
/home/mohan


In [3]:
def get_open_normalised_prices(df, start, end):
    
    print('************************************************************************')
    df1 = pd.DataFrame(df["High"]/df["Open"])
    df1.columns=["H/O"]
    df1["L/O"] = df["Low"]/df["Open"]
    df1["C/O"] = df["Close"]/df["Open"]
 
    return df1

# create_follow_cluster_matrix

In [4]:
def create_follow_cluster_matrix(data,k):
    print('************************************************************************')
    data["ClusterTomorrow"] = data["Cluster"].shift(-1)
    data.dropna(inplace=True)
    data["ClusterTomorrow"] = data["ClusterTomorrow"].apply(int)
#     print("Tesssssssssssttttttttttttttt")
#     print(zip(data["Cluster"], data["ClusterTomorrow"]))
    data["ClusterMatrix"] = list(zip(data["Cluster"], data["ClusterTomorrow"]))
    cmvc = data["ClusterMatrix"].value_counts()
    clust_mat = np.zeros( (k, k) )
    for row in cmvc.iteritems():
#         print(row[1]*100.0/len(data))
        clust_mat[row[0]] = row[1]*100.0/len(data)
    print("Cluster Follow-on Matrix:")
    print(clust_mat)
    return clust_mat,cmvc

In [5]:
def get_indicators(stocks, period):
#     for i in stocks:
        print("***********************************************************************")
        stocks.columns = [s.lower() for s in stocks.columns]
        print("Adding Features: Started  ")
        features = pd.DataFrame(SMA(stocks, timeperiod=10))
        features.columns = ['sma_10']
        features['NATR']=pd.DataFrame(NATR(stocks, timeperiod=14))
        features = pd.concat([features,STOCHF(stocks, fastk_period=14, fastd_period=3)], axis=1)
        features['willr'] = pd.DataFrame(WILLR(stocks, timeperiod=14))
        features['rsi'] = pd.DataFrame(RSI(stocks, timeperiod=14))
        features['wma_10'] = pd.DataFrame(WMA(stocks,10))
        features['T3'] =pd.DataFrame(T3(stocks, timeperiod=5, vfactor=0))
        features['closePrice']=pd.DataFrame(stocks['close'].shift(-period))
        features['return_pct_change'] = ROC(stocks, timeperiod=period)
        features['return_pct_change'] = features['return_pct_change'].shift(-period)
        features['pct_change'] = features['return_pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
           
        return features

In [6]:
def create_dataset1(datasetReal,i):
    print(type(datasetReal))
    xdataset = datasetReal.copy()
    xdataset = xdataset[xdataset.columns[xdataset.columns !='closePrice']]
    dataset = np.array(xdataset)
    print("Total Number of features in trainig Dataset "+ str(xdataset.columns.size))
    dataX = [dataset[n] for n in range(len(dataset)-i)]
    return np.array(dataX),xdataset.columns.size

In [7]:
predictedClosePrice2 = []
frameshistory=[]
# model = []
def testingModel1(inputClusterone):
    
        starttime1= time.time()
              
        
        for i in range(1,8):
            print("Training the Neural Network for next "+ str(i)+" Prediction")
            starttime= time.time() 
            
            print("Copying the input Dataset ")
            inputdata = inputClusterone.copy()
            
            dataset1 = get_indicators(inputClusterone,i)
            if(i ==1):
                print("Getting the features for Test data ")
                testdata = dataset1.tail(1)
                joblib.dump(testdata, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'testdata2.pkl')

            print("Making The Training Dataset")
            traindataset = dataset1.head(len(dataset1)-2)

            trainY = np.array(traindataset['closePrice'])[i:]
            trainX, numbFeatures = create_dataset1(traindataset,i)
            
            
            json_file = open(str(stock)+'model'+str(i)+'.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            loaded_model = model_from_json(loaded_model_json)
            # load weights into new model
            modelname = str(stock)+'model'+str(i)+'.h5' 
            loaded_model.load_weights(modelname)
            print("Loaded model"+ str(i)+" from disk")

            # evaluate loaded model on test data
            loaded_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'mape'])
            

            print("Model Compiled")

            print("Testing on the Latest Trade Data ...")

#             trainfeature=traindataset.tail(len(traindataset)-1)
#             trainlabel= trainfeature['closePrice']

#             prediction = m.predict(trainX)
#             my_list = map(lambda x: x[0], prediction)
#             traintestlabel = pd.Series(my_list)

#             testdata = dataset1.tail(1)
            testdata = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'testdata2.pkl')
#             testdata = testDataLatestTrade
#             testexpectedlabel = testdata['closePrice']

            xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
            print("-----------------------------------------")
            print("Test record Given")
            print(xtest)
            print("----------------------------------")
            
            predictiontest = loaded_model.predict(np.array(xtest))
            
            
            temp_list = map(lambda x: x[0], predictiontest)
            testlabel = pd.Series(temp_list)
            print("**********************************************************")
#             print("Excpected Close Price "+ str( testexpectedlabel.values))
            print("Predicted Close Price  "+ str(testlabel[0]))
#             predictedClosePrice.append(testlabel[0])
            clf = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice2.pkl') 
            clf.append(testlabel[0])
            joblib.dump(clf, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice2.pkl')
            print("**********************************************************")
            print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
            
        print("Testing the Model Finished ...")
        print("Total Time Taken to Model %s seconds ---" % (time.time() - starttime1))
#         allrecord = pd.concat(frameshistory)
#         allrecord.index = range(1,len(allrecord)+1)
#         print("Final Model Evaluation Scores ")
#         print(allrecord)


In [8]:
def datadownload(start,end,stock):
    try:
        dow = web.DataReader(stock, "yahoo", start, end)
        dow.to_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+str(stock)+"data.csv")
    except (RuntimeError, TypeError, NameError,Exception):
                   print("Oops! Yahoo Data Service is not working ....")
                   print("Data Taken from last updated repository")
                   dow=pd.read_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+str(stock)+"data.csv")
        
    
    return dow
    

In [9]:
def knn(dow_norm,k):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(dow_norm)
    labels = km.labels_
    return labels,km

In [10]:
def getinformationCluster(cmvc,km):
    print("Ranking of Clusters ");
    print(str(cmvc))
    print('************************************************************************')
    print("Centroid for the K Means Clusters ")
    kmean = pd.DataFrame(km.cluster_centers_)
    kmean.columns=['X','Y','Z']
    print(kmean)
    print('************************************************************************')
    

In [11]:
def predictCluster(km,dow_norm,cmvc,clust_mat):
    print("Getting the Latest Trade OHLC Values for DOW 30 ") 
    testdoeN = dow_norm.copy()
    print("*****************************************8")
#     print(testdoeN.tail(5))
#     del testdoeN['Cluster']
#     print("Testtttttt")
    
    print(testdoeN.tail(1).values.reshape(-1,3))
    predictedCluster = km.predict(testdoeN.tail(1).values.reshape(-1,3))[0]
    print("Predicted Cluster of Last Trade OHLC Date "+ " "+ str(testdoeN.index[len(testdoeN)-1])+" "+ str(predictedCluster))
    predictedNextCluster = []
    for i,row in cmvc.iteritems():
        if(i[0]==predictedCluster):
            predictedNextCluster.append(i[1])
    print("Predicted Next Day Cluster would be : "+ str(predictedNextCluster))
    
    probabCluster=[]
    totalPrbablityfromPast=0
    for i in predictedNextCluster:
            totalPrbablityfromPast=totalPrbablityfromPast+clust_mat[predictedCluster][i]
    #         probabCluster.append(clust_mat[predictedCluster][i])
    print("Calulating the Relative Probablity ")
    for i in predictedNextCluster:
            probabCluster.append(clust_mat[predictedCluster][i]/totalPrbablityfromPast*100)
            
    nextClusterMatrix = pd.DataFrame(predictedNextCluster)
    nextClusterMatrix.columns=["Cluster"]
    nextClusterMatrix['Probability']=probabCluster
    
    print("Next Day CLuster and its Relative probablity from the Predicted Cluster : "+ str(predictedCluster))
    print(nextClusterMatrix)
    return nextClusterMatrix
    


In [12]:
def getRightClusteredData(nextClusterMatrix,dowtrain):
    print("Getting the filtered Past data where the maximum predicted range of the next DAY OHLC would be.. ")
    inputClusterone = dowtrain[dowtrain['Cluster']==nextClusterMatrix['Cluster'][0]]
    return inputClusterone
    

In [13]:
def runprogram(stock):
    # Obtain DOW 30  pricing data from Yahoo Finance
    print('************************************************************************')
    print("Step 1:Downloading the"+str(stock)+" data from Yahoo Finance ")
    start = datetime.datetime(2005, 1, 1)
    end =  time.strftime("%Y-%m-%d")
#     dow = web.DataReader("^DJI", "yahoo", start, end)
    dow = datadownload(start,end,stock)
    print("Status ::Data Download Finish....")
    print('************************************************************************')
    print("Step 2:Normalising the Data with respect to Daily Open Stock Price ")
    
    dow_norm = get_open_normalised_prices(dow, start, end)
    print("Number of Chosen Cluster : 5 ")
    
    k = 5
    print('************************************************************************')
    print("Step 3: KNN Process Started ...")
    
    # Deleting the last row from the KNN Dataset 
    traindowNorm = dow_norm.copy()
    traindowNorm.drop(traindowNorm.index[len(traindowNorm)-1])
    
    # Reatining the latest Trade info as normalised one
    testDataLatestTrade = dow_norm.tail(1)
    
    labels,km = knn(traindowNorm,k)
    
    dowtrain = dow.copy()
    dowtrain.drop(dowtrain.index[len(dowtrain)-1])
    dowtrain["Cluster"] = labels
    traindowNorm["Cluster"] = labels
    
    print("Status : KNN Process Finished ...")
    # Create and output the cluster follow-on matrix
    clust_mat,cmvc = create_follow_cluster_matrix(dowtrain,k)
    
    print('************************************************************************')
    
    getinformationCluster(cmvc,km)
    
    nextClusterMatrix = predictCluster(km,testDataLatestTrade,cmvc,clust_mat)
    
    inputClusterone = getRightClusteredData(nextClusterMatrix,dowtrain)
    
    inputClusterone.append(dow.tail(1))
    
    joblib.dump(predictedClosePrice2, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice2.pkl')
#     joblib.dump(inputClusterone, str(stock)+'inputClusterone.pkl')
#     p = Pool(20)
# #     frameshistory=[]
#     print(p.map(testingModel, range (1,4)))
    testingModel1(inputClusterone)
#     print("Testing the Model Finished ...")
#     allrecord = pd.concat(frameshistory)
#     allrecord.index = range(1,len(allrecord)+1)
#     print("Final Model Evaluation Scores ")
#     print(allrecord)
    # load json and create model
    clf = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice2.pkl')
    print("**************************************")
    print(clf)
    



In [14]:
stocknames=["AAPL","T","BAC","JNJ","XOM"]
        
for i in stocknames:
    stock = i  ## TECHNOLOGY   ## APPLE 
    runprogram(i)



# stock = "AAPL"  ## TECHNOLOGY   ## APPLE 
# runprogram("AAPL")
# stock = "T"   ## TELECOMMUNICATION ## AT&T Inc.
# runprogram("T")
# stock = "BAC"    ## FINANCE BANK OF AMERICA
# runprogram("BAC")
# stock = "JNJ" ## HEALTH CARE - ## JOHNSON AND JHONSON
# runprogram("JNJ")
# stock = "XOM"  ## ENERGY - ## Exxon Mobil Corporation
# runprogram("XOM")

************************************************************************
Step 1:Downloading theAAPL data from Yahoo Finance 
Status ::Data Download Finish....
************************************************************************
Step 2:Normalising the Data with respect to Daily Open Stock Price 
************************************************************************
Number of Chosen Cluster : 5 
************************************************************************
Step 3: KNN Process Started ...
Status : KNN Process Finished ...
************************************************************************
Cluster Follow-on Matrix:
[[  4.84429066   1.54136521   6.57439446   6.00817867   1.10097515]
 [  1.44699591   0.53475936   1.16388802   0.94369299   0.47184649]
 [  5.19031142   1.06951872   7.01478452  13.40044039   0.78641082]
 [  7.95847751   0.88078012  11.60742372  23.5608682    0.56621579]
 [  0.59767222   0.53475936   1.10097515   0.69204152   0.40893363]]
******************

**********************************************************
Predicted Close Price  152.364624023
**********************************************************
Time Taken to Model 0.154541969299 seconds ---
Training the Neural Network for next 7 Prediction
Copying the input Dataset 
***********************************************************************
Adding Features: Started  
Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Loaded model7 from disk
Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
                sma_10     NATR      fastk     fastd     willr        rsi  \
Date                                                                        
2017-08-15  153.576002  1.68613  96.547806  95.84528 -3.452194  72.708714   

               wma_10          T3  return_pct_change pct_change  
Date                                                             
2017-08-1

**********************************************************
Predicted Close Price  38.8201446533
**********************************************************
Time Taken to Model 0.242850065231 seconds ---
Training the Neural Network for next 5 Prediction
Copying the input Dataset 
***********************************************************************
Adding Features: Started  
Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Loaded model5 from disk
Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
            sma_10      NATR      fastk      fastd      willr        rsi  \
Date                                                                       
2017-08-14  38.474  1.390653  69.689147  63.557856 -30.310853  56.284922   

               wma_10         T3  return_pct_change pct_change  
Date                                                            
2017-08-14  38

Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
            sma_10      NATR      fastk      fastd      willr        rsi  \
Date                                                                       
2017-08-16    24.5  1.893048  25.161403  36.129073 -74.838597  48.520299   

               wma_10        T3  return_pct_change pct_change  
Date                                                           
2017-08-16  24.394909  24.44834          -2.273675          0  
----------------------------------
**********************************************************
Predicted Close Price  23.593837738
**********************************************************
Time Taken to Model 0.227069854736 seconds ---
Training the Neural Network for next 4 Prediction
Copying the input Dataset 
***********************************************************************
Adding Features: Started  
Making The Training Dataset
<class 'pandas.core.frame.D

Loaded model2 from disk
Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
             sma_10      NATR      fastk      fastd      willr        rsi  \
Date                                                                        
2017-08-15  132.661  1.213735  56.865667  54.029805 -43.134333  57.199181   

                wma_10          T3  return_pct_change pct_change  
Date                                                              
2017-08-15  132.945455  132.752351           0.592287          1  
----------------------------------
**********************************************************
Predicted Close Price  135.680084229
**********************************************************
Time Taken to Model 0.337870836258 seconds ---
Training the Neural Network for next 3 Prediction
Copying the input Dataset 
***********************************************************************
Adding Features: Started  
Making The Trainin

Loaded model1 from disk
Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
            sma_10      NATR      fastk      fastd      willr        rsi  \
Date                                                                       
2017-08-07  80.614  1.112773  29.629947  39.043262 -70.370053  41.605252   

               wma_10         T3  return_pct_change pct_change  
Date                                                            
2017-08-07  80.461455  80.667346          -0.249507          0  
----------------------------------
**********************************************************
Predicted Close Price  78.8392791748
**********************************************************
Time Taken to Model 0.387231111526 seconds ---
Training the Neural Network for next 2 Prediction
Copying the input Dataset 
***********************************************************************
Adding Features: Started  
Making The Training Dataset

In [15]:
clf = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice2.pkl')

In [16]:
clf 

[78.839279174804688,
 79.357353210449219,
 78.832962036132812,
 79.305854797363281,
 80.249687194824219,
 81.046875,
 78.652183532714844]

In [164]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='maiti.t', api_key='km8Kdfszic1Cu6ZAWiZJ')
init_notebook_mode(connected=True)
import datetime as DT

In [165]:
def datadownload1(start,end,stockname):
    try:
        print(stockname)
        dow = web.DataReader(stockname, "yahoo", start, end)
#         dow.to_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+str(stock)+"data.csv")
    except (RuntimeError, TypeError, NameError,Exception):
                   print("Oops! Yahoo Data Service is not working ....")
#                    print("Data Taken from last updated repository")
#                    dow=pd.read_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+str(stock)+"data.csv")
        
    
    return dow

In [166]:
stocknames=["AAPL","T","BAC","JNJ","XOM"]
today = DT.date.today()  
start = today -  DT.timedelta(days=30)
end =  time.strftime("%Y-%m-%d")
week =[]
for i in range(1,8):
    weeks = today + DT.timedelta(days=i)
    week.append(weeks.strftime("%Y-%m-%d"))
for i in stocknames:
    print(i)
    df = datadownload1(start,end,i)
    clf = joblib.load(fileDir+ '/stockMarketModels/'+str(i)+'/'+str(i)+'predictedClosePrice2.pkl')
    iplot({"data": [Scatter(x=df.index, y=df['Close'],name=str('ClosePrice')),
                Scatter(x=week, y=clf,name=str('Future CP'))],
       "layout": Layout(title=str(i)+"Close Price % ",
                         yaxis=dict(title='Close Price in $ -->  '),
                         xaxis=dict(title='Date--> '))
           })
    
 
    

    


AAPL
AAPL


T
T


BAC
BAC


JNJ
JNJ


XOM
XOM
